In [11]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pyspark.sql.functions as F
from pyspark.sql.types import StructField,IntegerType, StructType,StringType
from awsglue.dynamicframe import DynamicFrame

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)


- Na célula a seguir, adicione o código necessário para ler o conteúdo do arquivo _nomes.csv_ para um dataframe Spark.



In [12]:
dynamicFrame = glueContext.create_dynamic_frame.from_options(
    connection_type="s3",
    connection_options={"paths": ["s3://aylu-compasso/dados/nomes.csv"]},
    format="csv",
    format_options={
        "withHeader": True,
    },
)
df = dynamicFrame.toDF()
df = df.withColumn("total", df["total"].cast(IntegerType()))
df = df.withColumn("ano", df["ano"].cast(IntegerType()))

- Imprima o esquema do dataframe

In [3]:
print(df)

DataFrame[nome: string, sexo: string, total: int, ano: int]


- Na próxima célula, escreva o código necessário para alterar a caixa dos valores da coluna _nome_ para MAIÚSCULO

In [4]:
df.withColumnRenamed("nome", "NOME").printSchema()

root
 |-- NOME: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- ano: integer (nullable = true)


- Apresente a contagem de linhas contidas no dataframe

In [5]:
df.count()

1825433


- Agora, apresente a contagem de nomes, agrupando os dados do dataframe pelas colunas _ano_ e _sexo_. Ordene os dados de modo que o ano mais recente aparece como primeiro registro do dataframe.

In [6]:
(df.groupBy("ano","sexo","NOME")
     .count()
     .orderBy("ano", ascending=False)
     .show()
)

+----+----+---------+-----+
| ano|sexo|     NOME|count|
+----+----+---------+-----+
|2014|   F|   Aubrey|    1|
|2014|   F|    Ciara|    1|
|2014|   F|   Camila|    1|
|2014|   F| Caroline|    1|
|2014|   F|  Madelyn|    1|
|2014|   F|      Eva|    1|
|2014|   F|      Ivy|    1|
|2014|   F| Margaret|    1|
|2014|   F|     Lola|    1|
|2014|   F|   Teagan|    1|
|2014|   F|    Brynn|    1|
|2014|   F|    Katie|    1|
|2014|   F|    Nadia|    1|
|2014|   F|    Megan|    1|
|2014|   F|   Baylee|    1|
|2014|   F|  Mallory|    1|
|2014|   F|Francesca|    1|
|2014|   F|  Lorelei|    1|
|2014|   F|    Raven|    1|
|2014|   F|   Danica|    1|
+----+----+---------+-----+
only showing top 20 rows


- Qual foi o nome feminino com mais registro e em que ano?

In [13]:
df2 = df.filter(F.col("sexo").rlike("F")).orderBy("total", ascending=False).show(1)

+-----+----+-----+----+
| nome|sexo|total| ano|
+-----+----+-----+----+
|Linda|   F|99680|1947|
+-----+----+-----+----+
only showing top 1 row


- Qual foi o nome masculino com mais registro e em que ano?

In [14]:
df3 = df.filter(F.col("sexo").rlike("M")).orderBy("total", ascending=False).show(1)

+-----+----+-----+----+
| nome|sexo|total| ano|
+-----+----+-----+----+
|James|   M|94755|1947|
+-----+----+-----+----+
only showing top 1 row


- Apresente o total de registros (masculinos e femininos) para cada ano presente no dataframe. Considere apenas as primeiras 10 linhas, ordenadas pelo ano, de forma crescente.

In [15]:
df4 = df.groupBy("sexo", "ano").count().orderBy("ano").show(10)

+----+----+-----+
|sexo| ano|count|
+----+----+-----+
|   F|1880|  942|
|   M|1880| 1058|
|   M|1881|  997|
|   F|1881|  938|
|   F|1882| 1028|
|   M|1882| 1099|
|   M|1883| 1030|
|   F|1883| 1054|
|   M|1884| 1125|
|   F|1884| 1172|
+----+----+-----+
only showing top 10 rows


- Escreva o conteúdo do dataframe com os valores de nome em maiúsculo no S3. Atenção aos requisitos:
  - A gravação deve ocorrer no subdiretório _frequencia_registro_nomes_eua_ do path s3://\<BUCKET\>/lab-glue/
  - O formato deve ser JSON
  - O particionamento deverá ser realizado pelas colunas _sexo_ e _ano_ (nesta ordem)

In [21]:
df = df.withColumn("nome", F.upper(df.nome))

DynamicFrame = DynamicFrame.fromDF(df, glueContext, "DynamicFrame")


glueContext.write_dynamic_frame \
    .from_options(
        frame = DynamicFrame, 
        connection_type = "s3", 
        connection_options =  {"path":"s3://aylu-compasso/lab-glue/frequencia_registro_nomes_eua", "partitionKeys": ["sexo", "ano"]},
        format = "csv")

In [22]:
job.commit()